In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [13]:
import sqlite3
import numpy as np
from tqdm import tqdm
from pathlib import Path
import openslide
import time
import pickle
import cv2
from glob import glob

In [14]:
import torchvision.transforms as transforms

In [15]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *

from data_loader import *

from helper.object_detection_helper import *
from loss.RetinaNetFocalLoss import RetinaNetFocalLoss
from models.RetinaNet import RetinaNet

In [16]:
path = Path('/data/Datasets/EIPH-Cat/')

size = 1024
level = 0

files = []

In [6]:
filenames = glob(str(path/'*.svs'), recursive=True)
for filename in tqdm(filenames):

    slide_path = path / filename

    slide = openslide.open_slide(str(slide_path))
    level = level#slide.level_count - 1
    level_dimension = slide.level_dimensions[level]
    down_factor = slide.level_downsamples[level]

    files.append(SlideContainer(slide_path,[[0], [1]], level, size, size))

100%|██████████| 14/14 [00:00<00:00, 75.18it/s]


In [19]:
torch.load(Path(path)/ fname, map_location='cpu')

{'opt_func': functools.partial(<class 'torch.optim.adam.Adam'>, betas=(0.9, 0.99)),
 'loss_func': RetinaNetEIPHFocalLoss(
   (reg_pred_loss): L1Loss()
 ),
 'metrics': [PascalVOCMetric
  anchors: tensor([[-0.9688, -0.9688,  0.1500,  0.1500],
          [-0.9688, -0.9688,  0.1750,  0.1750],
          [-0.9688, -0.9688,  0.2250,  0.2250],
          ...,
          [ 0.9688,  0.9688,  0.2250,  0.2250],
          [ 0.9688,  0.9688,  0.3125,  0.3125],
          [ 0.9688,  0.9688,  0.3750,  0.3750]])
  size: 1024
  metric_names: ['AP-0', 'AP-1', 'AP-2', 'AP-3', 'AP-4']
  detect_thresh: 0.3
  nms_thresh: 0.3
  images_per_batch: 20],
 'true_wd': True,
 'bn_wd': True,
 'wd': 0.01,
 'train_bn': True,
 'model_dir': 'models',
 'callback_fns': [functools.partial(<class 'fastai.basic_train.Recorder'>, add_time=True),
  fastai.train.ShowGraph,
  callbacks.callbacks.BBMetrics],
 'cb_state': {},
 'model': RetinaNetEIPH(
   (encoder): Sequential(
     (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), p

In [7]:
fname = "pferd_0_1024_reg.pth"

state = torch.load(Path(path) / fname, map_location='cpu') \
    if defaults.device == torch.device('cpu') \
    else torch.load(Path(path) / fname)
model = state.pop('model')
mean = state['data']['normalize']['mean']
std = state['data']['normalize']['std']

In [17]:
model

RetinaNetEIPH(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [8]:
anchors = create_anchors(sizes=[(32,32)], ratios=[1], scales=[0.6, 0.7, 0.9, 1.25, 1.5])
detect_thresh = 0.1 
nms_thresh = 0.3
result_boxes = {}
result_regression = {}

In [10]:
def rescale_box(bboxes, size: Tensor):
    bboxes[:, :2] = bboxes[:, :2] - bboxes[:, 2:] / 2
    bboxes[:, :2] = (bboxes[:, :2] + 1) * size / 2
    bboxes[:, 2:] = bboxes[:, 2:] * size / 2
    bboxes = bboxes.long()
    return bboxes

In [11]:
debug_level = 1
with torch.no_grad():
    for slide_container in tqdm(files):

        size = state['data']['tfmargs']['size']
        result_boxes[slide_container.file.name] = []
        result_regression[slide_container.file.name] = []

        basepic = np.array(slide_container.slide.read_region(location=(0, 0),
                                                             level=debug_level,
                                                             size=slide_container.slide.level_dimensions[debug_level]))
        basepic = basepic[:, :, :3].astype(np.uint8)

        for x in range(0, slide_container.slide.level_dimensions[level][1] - 2 * size, 900): #int(size / 2)
            for y in range(0, slide_container.slide.level_dimensions[level][0] - 2 * size, 900): #int(size / 2)
                x_real = x  # * slide_container.down_factor, \
                y_real = y  # * slide_container.down_factor

                patch_ori = slide_container.get_patch(x, y)
                patch = pil2tensor(patch_ori / 255., np.float32)

                patch = transforms.Normalize(mean, std)(patch)

                class_pred_batch, bbox_pred_batch, _, regression_pred, bbox_regression_pred = model.eval()(
                    patch[None, :, :, :])
                for clas_pred, bbox_pred, reg_pred, box_reg_pred in zip(class_pred_batch, bbox_pred_batch,
                                                                        regression_pred, bbox_regression_pred):

                    result_regression[slide_container.file.name].append(
                        np.array([x_real, y_real, x_real + size, y_real + size, reg_pred]))
                    bbox_pred, scores, preds = process_output(clas_pred, bbox_pred, anchors, detect_thresh)

                    if bbox_pred is not None:
                        to_keep = nms(bbox_pred, scores, nms_thresh)
                        bbox_pred, preds, scores = bbox_pred[to_keep].cpu(), preds[to_keep].cpu(), scores[to_keep].cpu()
                        box_reg_pred = box_reg_pred[to_keep].cpu()

                        t_sz = torch.Tensor([size, size])[None].float()

                        bbox_pred = rescale_box(bbox_pred, t_sz)

                        patch_ori = patch_ori.astype(np.uint8)
                        for box, pred, score, bb_reg in zip(bbox_pred, preds, scores, box_reg_pred):
                            y_box, x_box = box[:2]
                            h, w = box[2:4]

                            result_boxes[slide_container.file.name].append(np.array([x_box + x_real, y_box + y_real,
                                                                                     x_box + x_real + w, y_box + y_real + h,
                                                                                     pred, score, bb_reg]))

                            cv2.rectangle(patch_ori, (int(x_box), int(y_box)), (int(x_box + w), int(y_box + h)),
                                          (0, 0, 255), 1)

                            y_box, x_box = box[:2] / slide.level_downsamples[debug_level]
                            h, w = box[2:4] / slide.level_downsamples[debug_level]
                            temp_x_real = x_real / slide.level_downsamples[debug_level]
                            temp_y_real = y_real / slide.level_downsamples[debug_level]

                            cv2.rectangle(basepic, (int(x_box + temp_x_real), int(y_box + temp_y_real)),
                                          (int(x_box + temp_x_real + w), int(y_box + temp_y_real + h)), (255, 0, 0), 1)

        #cv2.imwrite("/server/born_pix_cm/{}.png".format(slide_container.file.stem), basepic[:, :, [2, 1, 0]])
        pickle.dump(result_boxes, open(str(path/"inference_results_boxes_all.p"), "wb"))
        pickle.dump(result_regression, open(str(path/"inference_result_regression_all.p"), "wb"))



100%|██████████| 14/14 [3:17:38<00:00, 871.68s/it]